In [17]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from network import VG_nn

In [18]:
df = pd.read_csv("training_massive.csv")
features = ["sigma", "theta", "kappa", "strike", "T"]
param_range = {param: [min(df[param]), max(df[param])] for param in df.columns}
X = df[features].values
y = df["expected_payoff"].values

In [19]:
#correlation entre les differentes features et le payoff
print(df.corr()["expected_payoff"])

sigma              0.086090
theta             -0.020479
kappa              0.014748
strike            -0.952231
T                  0.184364
expected_payoff    1.000000
error              0.795546
Ntraj                   NaN
Name: expected_payoff, dtype: float64


In [20]:
X.shape, y.shape

((319966, 5), (319966,))

In [21]:
print(X[0], y[0])

[ 0.1  -0.3   0.1   1.15  0.25] 2.3e-05


In [22]:
class Net(torch.nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(5, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc4 = torch.nn.Linear(hidden_size, 1)
        self.out = torch.nn.Softplus()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.out(x)
        return x.squeeze()

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=4096, shuffle=True)

model = Net(200).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss(reduction='mean')
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)

cuda:0


In [24]:
for epoch in range(100):
    for i, (X_batch, y_batch) in enumerate(dataloader):
        optimizer.zero_grad()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    scheduler.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: {loss.item()}")

Epoch 0: 0.0040422468446195126
Epoch 10: 1.3373923138715327e-05
Epoch 20: 5.544646683119936e-06
Epoch 30: 3.1360254979517777e-06
Epoch 40: 2.654631089171744e-06
Epoch 50: 2.1403366190497763e-06
Epoch 60: 2.347921054024482e-06
Epoch 70: 1.946660177054582e-06
Epoch 80: 2.4974665393528994e-06
Epoch 90: 1.7106519862863934e-06
